# Let's Break a Binary Cipher

While ciphers in the past worked on letters, words and symbols, computers opened up new doors in cryptography by operating at a binary level. We're going to learn about a type of cipher that can be used to encrypt files at the binary level, and then see how an attacker could break it.

#### Discussion: where do you think you encounter encryption in your day-to-day life?

This activity is powered by Python code, but you can safely ignore it. Whenever you see grey code boxes like below, feel free to scroll right past, unless you're wanting to get a deeper understanding of how I built this. 

   #### Load some necessary add-ons

In [ ]:
from PIL import Image #To create images
from IPython import display #To display images
import secrets #To get random bytes
import ipywidgets as widgets
import uuid #To get random file names

#### Create some functions of our own

In [ ]:
#This function takes in a file and gives a bytes for it
def load_file_as_bytes(filename,max_len=-1):
    file_in = open(filename,'rb')
    byte_array = bytearray()
    byte_in = file_in.read(1)
    while byte_in != b"" and ((len(byte_array) < max_len) or (max_len == -1)):
        byte_array += byte_in
        byte_in = file_in.read(1)
        
    file_in.close()
    
    return byte_array
            
#This function creates a black-and-white picture of our bytes (black=0, white=1)
def draw_image_from_byte_array(b_array,filename="imgout.png",width=2048,height=2048):
    assert (width%8 == 0)
    
    if len(b_array) < (width * height//8):
        height = len(b_array)//(width//8)
    img = Image.new("1", (width,height))
    
    for row in range(height):
        for byte_index in range(width//8):  
            byte = b_array[row*(width//8) + byte_index]
            for bit_index in range(8):
                b = (byte >> (7-bit_index))%2               
                img.putpixel((byte_index*8 + bit_index,row),b)
                
    img.save(filename)

def present_binary(filename):
    width = width_slider.value
    height = height_slider.value
    b = load_file_as_bytes(filename,max_len=width*height)
    tmp_filename = str(uuid.uuid4())+".png"
    draw_image_from_byte_array(b,tmp_filename,width,height)
    return display.Image(tmp_filename,width=width*zoom_slider.value,height=height*zoom_slider.value)

<a id='skip_to_start'></a>
## Look at files in binary

We can visualise the binary data to give us a better understanding of the file structure. This can help in understanding what the file type is. The widgets below will let you see the start of several different file types. Changing the width and height sliders below might help you see more meaningful structures in the files. Each pixel is a bit, with black being 0, and white being 1.

In [ ]:
file_dict = {
    "Image":"image.bmp",
    "Text":"alice.txt",
    "ZIP":"example.zip",
    "Program":"program.exe",
    "PDF":"puzzles.pdf",
    "Song":"song.wav"
}

def button_present(_arg):
    out.clear_output()
    out.append_display_data(present_binary(path_to_samples+file_dict[rdbuttons.value]))
    
width_slider = widgets.IntSlider(value=256,min=64,max=1024,step=8,description='Width')
height_slider = widgets.IntSlider(value=256,min=64,max=1024,description='Height')
zoom_slider = widgets.FloatSlider(value=1.0,min=0.5,max=4.0,description='Zoom factor')
path_to_samples = "Sample Files/"
out = widgets.Output()

button = widgets.Button(description="Analyse!")
button.on_click(button_present)

rdbuttons = widgets.RadioButtons(
    options=file_dict.keys(),
    description='File type:'
)

widgets.VBox([width_slider,height_slider,zoom_slider,rdbuttons,button,out])

Each file type could be storing its data in different ways.

#### Discussion: What structures could you see? Did some widths make these more evident?

## Look at random

In some sense, the purpose of encryption is to make the encrypted data look as random as possible. We'll see later that any structure left in the encrypted data might be exploitable to find patterns that help us break the cipher.

Humans are pretty bad judges of random, often seeing meaning and structure where there isn't any. Thankfully, computers can be very good at being unbiased. Press the button below to generate a giant block of computer random.

In [ ]:
def random_present(arg):
    with out2:
        out2.clear_output()
        random_bytes = secrets.token_bytes(512*512)
        draw_image_from_byte_array(random_bytes,filename="random.png",width=512,height=512)
        out2.append_display_data(display.Image("random.png"))


out2 = widgets.Output()
button2 = widgets.Button(description="Generate random!")
button2.on_click(random_present)

widgets.VBox([button2,out2])

#### Discussion: How does this compare to the files we saw earlier? Did one of the files look particularly random before?

With that settled, let's move on to a binary operation we'll use.

## Exclusive-Or (XOR)

XOR (Exclusive-or) (symbol ⊕) is an operation, like addition or subtraction. In fact, it's a lot like addition in binary, except we ignore any carries.

That is: 1 ⊕ 1 = 0, 1 ⊕ 0 = 1, and 0 ⊕ 0 = 0.

Another way to think about it is A ⊕ B = 1 if A and B are different bits, and A ⊕ B = 0 if A and B are the same bit.

If there are multiple units, like in the case of a byte, we can treat them all individually.

For example, 111 ⊕ 101 = 010.

#### Question: The opposite of addition is subtraction: if we start with a number, add X, and then subtract X, we are back to the starting number. What is the opposite of XOR?

## Learn XOR mask crypt

XOR is a fairly important building block of modern cryptography. It can be as simple as taking a block of secret random and XORing it with your message. If this is done with some caveats (like the secret block as long as the message, and never used again) then the system is proven to be 100% secure. 

Let's walk through this encryption to see what effect is has on the binary.

## Encrypt files

Let's get the computer to generate a secret key, then encrypt the text file.

Choose a key length for us to use. In this case, the longer the key the more secure it will be, and at some point the attack we use will stop working. Note that this is not always a reliable assumption!

We'll use this key again later when we look at breaking the cipher, so remember to come back up here if you want to change it!

In [ ]:
global my_key

key_slider = widgets.IntSlider(value=16,min=1,max=128,description='Key Length')

def encrypt(b,k):
    bytes_out = bytearray([])
    i = 0
    for byte in b:
        bytes_out.append(byte ^ k[i%len(k)])
        i += 1
    return bytes_out

keytext = widgets.Output(value="---", description = "Key:")
gen_button = widgets.Button(description="Generate key")

def generate_key(b):
    global my_key
    my_key = secrets.token_bytes(key_slider.value)
    with keytext:
        keytext.clear_output()
        print("Your key is:",''.join(["%02x"%x for x in my_key]))

gen_button.on_click(generate_key)
widgets.VBox([key_slider,gen_button,keytext])

## Encrypt the files

Let's see how the file looks after we encrypt it. We'll also take our secret file and encrypt it now. Press the 'Prepare files' button. This step will take a little while but you'll know it's done when it shows you pictures of the binary files.

In [ ]:
global img_encrypted
global enc_secret
def prepare(arg):
    global img_encrypted
    global enc_secret
    b_secret = load_file_as_bytes("Sample Files/secret.txt")
    enc_secret = encrypt(b_secret,my_key)

    b_test = load_file_as_bytes("Sample Files/image.bmp")
    draw_image_from_byte_array(b_test)
    img1 = display.Image("imgout.png",width=512,height=512)

    img_encrypted = encrypt(b_test,my_key)
    draw_image_from_byte_array(img_encrypted,"encrypted.png")
    img2 = display.Image("encrypted.png",width=512,height=512)

    prepare_output.clear_output()
    with prepare_output:
        print("The original binary file:")
        display.display(img1)

        print("The encrypted binary file:")
        display.display(img2)
        
        print("Ready to move on!")

prepare_encryption_button = widgets.Button(description="Prepare files")
prepare_encryption_button.on_click(prepare)
prepare_output = widgets.Output()
widgets.VBox([prepare_encryption_button,prepare_output])

#### Discussion: What does the encrypted data look like now?

We can see that where the file was particularly structured, the encrypted version is still a bit structured, but in the less structured portion, it looks essentially random. This will also depend on how long you chose to make your key.

## Decrypt files

Now we have to write a decrypt function. This should take the encrypted data and the key, and undo the operations of the encrypt function.

#### Discussion: Isn't it convenient that the encryption function and decryption function are exactly the same?

In [ ]:
def decrypt(b,k):
    return encrypt(b,k) 

## Look at files

While we still know the key, let's press the button to decrypt our file and make sure it's all working.

In [ ]:
global img_decrypted
def show_files(arg):
    global img_decrypted
    show_output.clear_output()
    with show_output:
        img_decrypted = decrypt(img_encrypted,my_key)
        draw_image_from_byte_array(img_decrypted,"decrypted.png")
        print("The decrypted binary file:")
        display.display(display.Image("decrypted.png",width=512,height=512))
    
show_button = widgets.Button(description='Show file')
show_button.on_click(show_files)
show_output = widgets.Output()

widgets.VBox([show_button,show_output])

## Convert from bytes back into a file

And lastly, press the button below to convert it into readable text.

In [ ]:
button = widgets.Button(description="Check decrypt!")
output = widgets.Output()

display.display(button, output)

def on_button_clicked(b):
    output.clear_output()
    output.append_display_data(display.Image(img_decrypted))

button.on_click(on_button_clicked)

If a lovely image appeared above, then the decryption function worked as planned.

# Now for _CRYPTANALYSIS_

We have to be able to understand the mind of the attacker to see if our system really is secure, so we're going to try and crack another file.

In practice, it's good to assume your attacker knows how your encryption system works, but they do not know the secret key used. If we design our system with this in mind, we only have to think about protecting our key, and not have to worry about other things (like our encryption program itself) being kept secret.

Using our XOR cipher, we'll see that patterns and structure in the encrypted files can be enough to recover the key. We'll be encrypting a text document (language is very structured).

#### Discussion: How could patterns and structure in the underlying message, or in the cipher, help us break it?


## Finding the key length

Our first step will be determining the key length. This will help us impose structure on the data to give us more insight into the secret key.

If this coincidence is strong enough, we'll be able to use it to recover the key length. What we're doing is looking for repeats in the cipher text, that are at least 4 bytes long, and appear at least 4 times.

In [ ]:
global putative_key_length
def find_repeated_segments(b,length=4,min_repeats=4):
    found = {}
    for i in range(len(b) - length + 1):
        segment = b[i:i+length]
        if segment not in found:
            found[segment] = []
        found[segment].append(i)
    
    repeated = {}
    for segment in found:
        if len(found[segment]) >= min_repeats:
            repeated[segment] = found[segment]
    
    return repeated 

def differences_in_array(a):
    diffs = []
    for i in range(len(a) - 1):
        diffs.append(a[i+1] - a[i])
    return diffs

def gcd_of_differences(d):
    def gcd(a,b):
        while b>0:
            a,b = b,a%b
        return a
    g = d[0]
    for i in range(1, len(d)):
        g = gcd(g,d[i])
    return g

def find_length(arg):
    global putative_key_length
    rs = find_repeated_segments(bytes(enc_secret))
    if len(rs.keys()) == 0:
        with key_length_output:
            print("Oh no! There was not enough structure to break this. See a helper!")
            return
    rsk = list(rs.keys())[0]

    #Calculate how far apart they occur
    d = differences_in_array(rs[rsk])

    #Find the largest key that could line up with all of these occurrences
    putative_key_length = gcd_of_differences(d)
    key_length_output.clear_output()
    with key_length_output:
        print("Putative key length:",putative_key_length)

key_length_button = widgets.Button(description="Find Length")
key_length_button.on_click(find_length)
key_length_output = widgets.Output()

widgets.VBox([key_length_button,key_length_output])

## Get best key guess

Now we're choosing the key that makes our decrypted file have the most ordinary structure. This code will do this by choosing each individual key byte to be the one that maximises the occurrences of what we expect to be the most common byte. Each byte corresponds to exactly one character, so the question is really what is the most common character?

#### Discussion: What would we expect the most common character in a text file to be?

In [ ]:
#This function returns the 'majority' byte of a given bytearray
def majority(ba):
    results = {}
    
    for b in ba:
        if b not in results:
            results[b] = 0
        results[b] += 1
        
    return max(results.keys(),key=lambda b : results[b])
    
        
#This function breaks the bytearray into sub-sequences of every 'cutsize'th entry.
def slice(ba,cutsize):
    subsequences = {}
    for i in range(cutsize):
        subsequences[i] = bytearray()
    i = 0
    while i < len(ba):
        subsequences[i%cutsize].append(ba[i])
        i += 1
    return subsequences

    
def find_key(arg):
    if len(common_character_field.value) != 1:
        return
    plain_majority = ord(common_character_field.value)
    cuts = slice(enc_secret,putative_key_length)
    majority_of_cuts = bytearray()
    for c in cuts:
        majority_of_cuts.append(majority(cuts[c]) ^ plain_majority)

    global suspect_key
    suspect_key = bytes(majority_of_cuts)
    find_key_output.clear_output()
    with find_key_output:
        print("We have guessed our key length is %d bytes (%d bits) (let's hope we're correct!)"%(putative_key_length,8*putative_key_length))
        print("We guessed the most common character is '%s'."%common_character_field.value)
        print("Key guess:",''.join(["%02x"%x for x in suspect_key]))

common_character_field = widgets.Text(description="Character:")
find_key_button = widgets.Button(description="Find Key")
find_key_output = widgets.Output()

find_key_button.on_click(find_key)
widgets.VBox([common_character_field,find_key_button,find_key_output])

If our key guess was good, the decrypted text should be mostly readable. Press the button below to find out! If it still looks really messy, try a different character and Find Key again.

In [ ]:
global secret_decrypted
def attempt_decrypt(arg):
    global secret_decrypted
    dec_secret = decrypt(enc_secret, suspect_key)
    secret_decrypted = ''.join([chr(x) for x in dec_secret])
    attempt_output.clear_output()
    with attempt_output:
        print(secret_decrypted)
    
attempt_button = widgets.Button(description="Attempt Decrypt")
attempt_button.on_click(attempt_decrypt)
attempt_output = widgets.Output()

widgets.VBox([attempt_button,attempt_output])

## Make final adjustments to the key.

Great, it's a fusty old book on ciphers. With that context in mind, we can make the last adjustments manually to recover the key (or at least decide the information within is not worth pursuing). It's possible that the system worked perfectly first time, though this is less likely with a longer key.

Use the function below to make changes to individual bytes of the key to fit our expectations.

Looking at the left column (that is the index of the key byte), and the corresponding character is the resulting decrypted text. In the character field, put in the single character you believe should be in that place, and press change. Keep going until you are satisfied with the decryption.

In [ ]:
def swap_key_characters(arg):
    global suspect_key
    global secret_decrypted
    global putative_key_length
    
    if len(character_input.value) != 1:
        return
    
    i = index_input.value
    
    suspect_key = bytes([suspect_key[i%putative_key_length] ^ ord(secret_decrypted[i]) ^ ord(character_input.value) if (i%putative_key_length) == j else suspect_key[j] for j in range(len(suspect_key))])

    dec_secret = decrypt(enc_secret, suspect_key)
    secret_decrypted = ''.join([chr(x) for x in dec_secret])
    
    key_list.clear_output()
    with key_list:
        offset = 0
        for i in range(offset*putative_key_length,(offset+1)*putative_key_length):
            print("% 4d %s"%(i,secret_decrypted[i]))
    final_decrypt.clear_output()
    with final_decrypt:
        print(secret_decrypted)

    key_list.clear_output()
    with key_list:
        for i in range(putative_key_length):
            print("% 4d %s"%(i,secret_decrypted[i]))
    final_decrypt.clear_output()
    with final_decrypt:
        print(secret_decrypted)
        
def ready(arg):
    key_list.clear_output()
    with key_list:
        for i in range(putative_key_length):
            print("% 4d %s"%(i,secret_decrypted[i]))
    final_decrypt.clear_output()
    with final_decrypt:
        print(secret_decrypted)
        
    with ready_output:
        ready_output.clear_output()
        display.display(widgets.VBox([index_input,character_input,input_button,widgets.HBox([key_list,final_decrypt])]))    
       
index_input = widgets.IntText(description='Index:')
character_input = widgets.Text(description='Character:')
input_button = widgets.Button(description="Change")
input_button.on_click(swap_key_characters)
final_decrypt = widgets.Output()
key_list = widgets.Output(layout=widgets.Layout(min_width='8%')) 

ready_button = widgets.Button(description="Ready!")
ready_button.on_click(ready)
ready_output = widgets.Output()
with ready_output:
    display.display(ready_button)
    
ready_output